# Data Visualization

### Imports

In [1]:
import sys
import os
import numpy
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
import gillespy2
from gillespy2 import TauHybridSolver

### Model Instantiation

Model must include rates, species, and reactions

In [ ]:
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.5)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='Substrate', initial_value=301)
            B = gillespy2.Species(name='Enzyme', initial_value=120)
            C = gillespy2.Species(name='Enzyme_Substrate_Complex', initial_value=0)
            D = gillespy2.Species(name='Product', initial_value=0)
            self.add_species([A, B, C, D])
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            self.add_reaction([r1,r2,r3])
            self.timespan(numpy.linspace(0,100,101))

In [ ]:
model = MichaelisMenten()


### Run Model and plot

In [ ]:
%time results = model.run(solver=TauHybridSolver())

model.run creates a Results object with a list of Trajectory objects. Results makes use of .plot() to make matplotlib graphs as well as .plotplotly() in order to create more advanced interactable plotly graphs. Plotly graphs can only be displayed in Jupyter Notebooks.

To pass an argument to plotly's Layout constructor, simply include it in the call to Results.plotplotly(). See the parameter xaxis_type='log' in the subsequent cell.

In [ ]:
results.plot(yscale='log',xaxis_label='Custom X Label')

In [ ]:
results.plotplotly(yaxis_type='log')

Results.plot() and Results.plotplotly() take optional arguments for custom labels and titles. The included_species_list may be used to limit which species are included in the graph.

Additionally, plot() and plot_std_dev_range() (the matplotlib graphers) include the optional figsize keyword which takes a tuple of the form figsize = (width,height).

Lastly, plot() has the style keyword which can be used to define which matplotlib style the graph uses. A complete list of possible styles can be found at https://matplotlib.org/3.1.1/gallery/style_sheets/style_sheets_reference.html. 

In [ ]:
results.plot(xaxis_label ="Custom X", yaxis_label ="Custom Y", title = "Graph with custom labels", show_legend=True,figsize= (8,8),included_species_list=['Product','Substrate'])

plot() also includes the save_png keyword which may be set to True in order to save the matplotlib graph as a png file. plotplotly() does not support the save_png keyword however plotly graphs may be saved inline by clicking on the camera icon in the top right. 

### Ploting multiple runs 

The number_of_trajectories specifies how many times you would like to run the solver. When model.run is ran with multiple trajectories, the Results object returned contains a list of all resulting trajectories.

In [ ]:
%time results_many_trajectories = model.run(solver=TauHybridSolver(),number_of_trajectories=5)

By default, multiple trajectories are displayed in a single graph.

In [ ]:
results_many_trajectories.plot(title="Many trajectories on one plot")

In [ ]:
results_many_trajectories.plotplotly(title="Many trajectories on one plot")

Results can use the average_ensemble function to create a single results dictionary of the average of the trajectories. 

In [ ]:
average_results = results_many_trajectories.average_ensemble()
average_results.plot(title="The average of those trajectories")

Results can use the stddev_results function to create a single results dictionary of the standard deviation of the trajectories. 

In [ ]:
stddev_results = results_many_trajectories.stddev_ensemble()
stddev_results.plot(title="The standard deviation of trajectories for each time")

Standard deviation range and mean may be visualized with the plot_std_dev_range() and plotplotly_std_dev_range() as well as using plotplotly_mean_stdev and plot_mean_stdev

In [ ]:
results_many_trajectories.plot_std_dev_range()


In [ ]:
results_many_trajectories.plotplotly_std_dev_range()

In [ ]:
results_many_trajectories.plot_mean_stdev()

In [ ]:
results_many_trajectories.plotplotly_mean_stdev()

Results objects are lists of Trajectory objects, which are dictionaries containing lists of species population data. Data from the first Trajectory in the list may be accessed directly from the Results object. Custom plots may be created using matplotlib as in the example below.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18,10))
plt.title("Michaelis-Menten: Tau Hybrid Solver")
plt.xlabel("Time")
plt.ylabel("Population")
plt.plot(results['time'],results['Substrate'],'r', label='Substrate')
plt.plot(results['time'],results['Enzyme'],'y', label='Enzyme')
plt.plot(results['time'],results['Enzyme_Substrate_Complex'],'g', label='Enzyme-Substrate Complex')
plt.plot(results['time'],results['Product'],'b', label='Product')
plt.plot([0],[11])
plt.legend(loc='best')



### A more complete list of arguments for each of the above methods can be found [in the documentation](https://stochss.github.io/GillesPy2/classes/gillespy2.core.html#gillespy2.core.results.Results).